# LSSTCam Test Calibrations
Author: Alex Broughton


# Detectors:
- 76  R20_S11 ITL (good)
- 94  R22_S11 E2V (good)
- 117 R30_S00 ITL (dead amp C10)
- 173 R42_S02 ITL (vampire infestation)

# 0. Setup

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

from lsst.daf.butler import Butler
from lsst.obs.lsst import LsstCam

camera = LsstCam.getCamera()

color = cm.tab20(np.linspace(0, 1, 16))

collections = [
    "u/abrought/LSSTCam/calib/DM-49175", 
    "u/abrought/LSSTCam/calib/DM-49175/test/linearizerGen.20250309a",
    # "u/abrought/LSSTCam/calib/DM-49175/test/ptcGen.20250309a",
    "u/abrought/LSSTCam/calib/DM-49175/test/darkGen.20250309a",
    "u/abrought/LSSTCam/calib/DM-49175/test/biasBootstrapGen.20250309a/run1",
    "u/abrought/LSSTCam/calib/DM-49175/test/darkBootstrapGen.20250309a/run1",
    "u/abrought/LSSTCam/calib/DM-49175/test/flatBootstrapGen.20250309a/run1",
]
butler = Butler("/repo/main", collections=collections)
registry = butler.registry

detIds = [76, 94, 117, 173]

In [ ]:
# import os
# for detector in camera:
#     detId = detector.getId()
#     detName = detector.getName()
#     raft = detName[0:3]
#     bay = detName[4:]
#     os.makedirs(f"./plots/{raft}", exist_ok=True)
#     os.makedirs(f"./plots/{raft}/bay", exist_ok=True)

# 1. Bias Bootstrap

In [ ]:
refs = list(butler.query_datasets(
    "biasBootstrap", 
    where=f"instrument='LSSTCam' and detector in {str(tuple(detIds))}", 
    collections=collections))

for ref in refs:
    plt.figure()
    img = butler.get("biasBootstrap", dataId=ref.dataId, collections=collections)
    for i, amp in enumerate(camera[ref.dataId['detector']].getAmplifiers()):
        bbox = amp.getBBox()
        vals = img.image[bbox].array.ravel()
        vals = vals[(vals>-10.) * (vals<10.)]
        _ = plt.hist(vals, bins=100, histtype='step', color=color[i], label=amp.getName())
    
    plt.yscale('log')
    plt.xlabel("adu")
    plt.legend(ncol=2)
    plt.title(f"BIAS BOOTSTRAP {camera[ref.dataId['detector']].getName()}, {ref.dataId['detector']}")
    plt.savefig(f"./plots/biasBootstrap_{ref.dataId['detector']}.png", bbox_inches='tight')
    #plt.close()

# 2. Dark Bootstrap

In [ ]:
refs = list(butler.query_datasets(
    "darkBootstrap", 
    where=f"instrument='LSSTCam' and detector in {str(tuple(detIds))}", 
    collections=collections))

for ref in refs:
    plt.figure()
    img = butler.get("darkBootstrap", dataId=ref.dataId, collections=collections)
    for i, amp in enumerate(camera[ref.dataId['detector']].getAmplifiers()):
        bbox = amp.getBBox()
        vals = img.image[bbox].array.ravel()
        vals = vals[(vals>-10.) * (vals<10.)]
        _ = plt.hist(vals, bins=100, histtype='step', color=color[i], label=amp.getName())
    
    plt.yscale('log')
    plt.xlabel("adu")
    plt.legend(ncol=2)
    plt.title(f"dark BOOTSTRAP {camera[ref.dataId['detector']].getName()}, {ref.dataId['detector']}")
    plt.savefig(f"./plots/darkBootstrap_{ref.dataId['detector']}.png", bbox_inches='tight')
    #plt.close()

# 3. Flat Bootstrap

In [ ]:
refs = list(butler.query_datasets(
    "flatBootstrap", 
    where=f"instrument='LSSTCam' and detector in {str(tuple(detIds))}", 
    collections=collections))

for ref in refs:
    plt.figure()
    img = butler.get("flatBootstrap", dataId=ref.dataId, collections=collections)
    for i, amp in enumerate(camera[ref.dataId['detector']].getAmplifiers()):
        bbox = amp.getBBox()
        vals = img.image[bbox].array.ravel()
        #vals = vals[(vals>-10.) * (vals<10.)]
        _ = plt.hist(vals, bins=100, histtype='step', color=color[i], label=amp.getName())
    
    plt.yscale('log')
    plt.xlabel("adu")
    plt.legend(ncol=2)
    plt.title(f"FLAT BOOTSTRAP {camera[ref.dataId['detector']].getName()}, {ref.dataId['detector']}")
    plt.savefig(f"./plots/flatBootstrap_{ref.dataId['detector']}.png", bbox_inches='tight')
    #plt.close()

# 4. Bias v. Dark

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

from lsst.daf.butler import Butler
from lsst.obs.lsst import LsstCam

# Camera object
camera = LsstCam.getCamera()

color = cm.tab20(np.linspace(0, 1, 16))

butler = Butler("/repo/main")
registry = butler.registry
refs = list(butler.query_datasets(
    "biasBootstrap", 
    where=f"instrument='LSSTCam' and detector in {str(tuple(detIds))}", 
    collections = collections
))
refs2 = list(butler.query_datasets(
    "darkBootstrap", 
    where=f"instrument='LSSTCam' and detector in {str(tuple(detIds))}", 
    collections = collections
))

for ref, ref2 in zip(refs, refs2):
    plt.figure()
    img = butler.get(
        "biasBootstrap", 
        dataId=ref.dataId, 
        collections=collections
    )
    img2 = butler.get(
        "darkBootstrap", 
        dataId=ref.dataId, 
        collections=collections
    )
    
    fig, axes = plt.subplots(2, 8, figsize=(16,4))
    fig.suptitle(camera[ref.dataId['detector']].getName())
    axs=axes.ravel()
    for i, amp in enumerate(camera[ref.dataId['detector']].getAmplifiers()):
        ax = axs[i]
        bbox = amp.getBBox()
        vals = img.image[bbox].array.ravel()
        vals2 = img2.image[bbox].array.ravel() * 15.0 # scaled by 15s
        #vals = vals[(vals>-15.) * (vals<15.)]
        ax.hexbin(vals, vals2, bins='log', gridsize=(100,100),extent=(-15,15,-15, 15))
        ax.set_xlabel("BIAS BOOTSTRAP")
        ax.set_ylabel("DARK BOOTSTRAP * 15s")
        ax.set_xlim(-15., 15.)
        ax.set_ylim(-15., 15.)
        ax.set_title(amp.getName())
        ax.set_aspect('equal')
    plt.tight_layout()
    plt.savefig(f"./plots/biasVdark_{ref.dataId['detector']}.png", bbox_inches="tight")
    #plt.close()

# 5. Linearizer

In [ ]:
refs = list(butler.query_datasets(
    "linearizer",
    where=f"instrument='LSSTCam' and detector in {str(tuple(detIds))}", 
    collections=collections
))

linTurnoffs = []
for ref in refs: 
    lin = butler.get("linearizer", dataId=refs[0].dataId, collections=collections)
    ptc = butler.get("linearizerPtc", dataId=refs[0].dataId, collections=collections)
    
    fig, axes = plt.subplots(4, 4, figsize=(12,12))
    fig.suptitle(camera[ref.dataId['detector']].getName())
    axs=axes.ravel()
    for i, amp in enumerate(camera[ref.dataId['detector']].getAmplifiers()):
        ampName = amp.getName()
        ax = axs[i]
        ax.set_title(ampName)
        ax.scatter(ptc.photoCharges[ampName], ptc.rawMeans[ampName], s=1)
        ax.axhline(lin.linearityTurnoff[ampName], linestyle="--", color="k", alpha=0.5)
        linTurnoff = float(lin.linearityTurnoff[ampName])
        linTurnoffs.append(linTurnoff)
        ax.text(
            0, lin.linearityTurnoff[ampName]*.8, 
            f"Linearity Turnoff:\n{round(linTurnoff, 1)} adu",
        )
        
        ax.ticklabel_format(style='sci', axis='both', scilimits=(0, 0), useMathText=True)
        ax.set_xlabel("Integrated Photodiode\nCurrent")
        ax.set_ylabel("Raw Means (adu)")
    plt.tight_layout()
    plt.savefig(f"./plots/photocurrentVsrawMeans_{camera[ref.dataId['detector']].getId()}.png", bbox_inches='tight')
    #plt.close()


    fig, axes = plt.subplots(4, 4, figsize=(12,12))
    fig.suptitle(camera[ref.dataId['detector']].getName())
    axs=axes.ravel()
    for i, amp in enumerate(camera[ref.dataId['detector']].getAmplifiers()):
        ampName = amp.getName()
        ax = axs[i]
        ax.set_title(ampName)
        ax.scatter(ptc.photoCharges[ampName], lin.fitResiduals[ampName]/ptc.rawMeans[ampName], s=1)
        ax.axhline(0, linestyle="--", color="k", alpha=0.5)
        ax.ticklabel_format(style='sci', axis='x', scilimits=(0, 0), useMathText=True)
        ax.set_xlabel("Integrated Photodiode\nCurrent")
        ax.set_ylabel("fitResiduals / rawMeans")
    plt.tight_layout()
    plt.savefig(f"./plots/fitResidualsVsrawMeans_{camera[ref.dataId['detector']].getId()}.png", bbox_inches='tight')
    #plt.close()

In [ ]:
plt.hist(np.array(linTurnoffs), histtype="step")
plt.ticklabel_format(style='sci', axis='x', scilimits=(0, 0), useMathText=True)
plt.xlabel("Linearity Turnoffs (adu)")
plt.savefig(f"./plots/linearity_turnoffs_hist.png", bbox_inches='tight')
#plt.close()

# 5. PTC

In [ ]:
refs = list(butler.query_datasets(
    "ptc",
    where=f"instrument='LSSTCam' and detector in {str(tuple(detIds))}", 
    collections=collections
))

ptcTurnoffs = []
empiricalN00s = []
n00s = []
gains = []
a00s = []

for ref in refs: 
    ptc = butler.get("ptc", dataId=ref.dataId, collections=collections)
    
    fig, axes = plt.subplots(4, 4, figsize=(12,12))
    fig.suptitle(camera[ref.dataId['detector']].getName())
    axs=axes.ravel()
    for i, amp in enumerate(camera[ref.dataId['detector']].getAmplifiers()):
        ampName = amp.getName()
        ax = axs[i]
        ax.set_title(ampName)
        ax.scatter(ptc.rawMeans[ampName], ptc.rawVars[ampName], s=1)
        ax.scatter(ptc.finalMeans[ampName], ptc.finalVars[ampName], s=1)
        
        ptcTurnoff = float(ptc.ptcTurnoff[ampName])
        ptcTurnoffs.append(ptcTurnoff)

        a00 = ptc.aMatrix[ampName][0][0] * 1e6
        a00s.append(a00)
        n00 = ptc.noise[ampName]
        n00s.append(n00)
        empiricalN00 = np.median(ptc.noiseList[ampName])
        empiricalN00s.append(empiricalN00)
        gain = ptc.gain[ampName]
        gains.append(gain)
        
        ax.axvline(ptcTurnoff, linestyle="--", color="k", alpha=0.5)
        ax.text(
            5e3, 4.5e4, 
            f"Turnoff: {round(ptcTurnoff, 1)} adu",
        )
        ax.text(
            5e3, 4.1e4, 
            f"Gain: {round(gain, 2)}",
        )
        ax.text(
            5e3, 3.7e4, 
            r"$a_{00}$: " + f"{round(a00, 2)}" + r"$\times 10^{-6}$",
        )
        ax.text(
            5e3, 3.3e4, 
            r"$n_{00}$: " + f"{round(n00, 2)} el",
        )
        
        ax.ticklabel_format(style='sci', axis='both', scilimits=(0, 0), useMathText=True)
        ax.set_xlabel("Signal (adu)")
        ax.set_ylabel(r"$C_{00}$ (adu$^2$)")
        # ax.set_xscale('log')#, linthresh=15000.0)
        # ax.set_yscale('log')#, linthresh=15.0)
        ax.set_ylim(0, 5.5e4)
        ax.set_xlim(0, 1.25e5)
    plt.tight_layout()
    plt.savefig(f"./plots/ptc_{camera[ref.dataId['detector']].getId()}.png", bbox_inches='tight')
    #plt.close()


In [ ]:
for ref in refs: 
    ptc = butler.get("ptc", dataId=ref.dataId, collections=collections)
    
    fig, axes = plt.subplots(4, 4, figsize=(12,12))
    fig.suptitle(camera[ref.dataId['detector']].getName())
    axs=axes.ravel()
    for i, amp in enumerate(camera[ref.dataId['detector']].getAmplifiers()):
        ampName = amp.getName()
        ax = axs[i]
        ax.set_title(ampName)
        
        ptcTurnoff = ptc.ptcTurnoff[ampName]

        good = ptc.expIdMask[ampName] * (ptc.rawMeans[ampName] < ptcTurnoff)
        data = ptc.rawVars[ampName][good] / ptc.rawMeans[ampName][good]

        ax.scatter(ptc.rawMeans[ampName][good], data, s=1)
        #ax.axhline(0, linestyle="--", color="k", alpha=0.5)
        ax.axvline(ptcTurnoff, linestyle="--", color="orange", alpha=0.5)
        
        ax.ticklabel_format(style='sci', axis='x', scilimits=(0, 0), useMathText=True)
        ax.set_xlabel("Signal (adu)")
        ax.set_ylabel(r"$C_{00} / \mu$")
        # ax.set_xscale('log')#, linthresh=15000.0)
        # ax.set_yscale('log')#, linthresh=15.0)
        ax.set_ylim(0.4, 0.8)
        ax.set_xlim(0, 1.0e5)
    plt.tight_layout()
    plt.savefig(f"./plots/C00_mu_{camera[ref.dataId['detector']].getId()}.png", bbox_inches='tight')
    #plt.close()


In [ ]:
for ref in refs: 
    ptc = butler.get("ptc", dataId=ref.dataId, collections=collections)
    
    fig, axes = plt.subplots(4, 4, figsize=(12,12))
    fig.suptitle(camera[ref.dataId['detector']].getName())
    axs=axes.ravel()
    for i, amp in enumerate(camera[ref.dataId['detector']].getAmplifiers()):
        ampName = amp.getName()
        ax = axs[i]
        ax.set_title(ampName)
        
        ptcTurnoff = ptc.ptcTurnoff[ampName]

        good = ptc.expIdMask[ampName] * (ptc.rawMeans[ampName] < ptcTurnoff)
        data = ptc.covariances[ampName][good][:,0,1] / ptc.rawMeans[ampName][good]

        ax.scatter(ptc.rawMeans[ampName][good], data, s=1)
        ax.axhline(0, linestyle="--", color="k", alpha=0.5)
        ax.axvline(ptcTurnoff, linestyle="--", color="orange", alpha=0.5)
        
        ax.ticklabel_format(style='sci', axis='x', scilimits=(0, 0), useMathText=True)
        ax.set_xlabel("Signal (adu)")
        ax.set_ylabel(r"$C_{01} / \mu$")
        # ax.set_xscale('log')#, linthresh=15000.0)
        # ax.set_yscale('log')#, linthresh=15.0)
        ax.set_ylim(-0.01, 0.03)
        ax.set_xlim(0, 1.0e5)
    plt.tight_layout()
    plt.savefig(f"./plots/C01_mu_{camera[ref.dataId['detector']].getId()}.png", bbox_inches='tight')
    #plt.close()


In [ ]:
for ref in refs: 
    ptc = butler.get("ptc", dataId=ref.dataId, collections=collections)
    
    fig, axes = plt.subplots(4, 4, figsize=(12,12))
    fig.suptitle(camera[ref.dataId['detector']].getName())
    axs=axes.ravel()
    for i, amp in enumerate(camera[ref.dataId['detector']].getAmplifiers()):
        ampName = amp.getName()
        ax = axs[i]
        ax.set_title(ampName)
        
        ptcTurnoff = ptc.ptcTurnoff[ampName]

        good = ptc.expIdMask[ampName] * (ptc.rawMeans[ampName] < ptcTurnoff)
        data = ptc.covariances[ampName][good][:,1,0] / ptc.rawMeans[ampName][good]

        ax.scatter(ptc.rawMeans[ampName][good], data, s=1)
        ax.axhline(0, linestyle="--", color="k", alpha=0.5)
        ax.axvline(ptcTurnoff, linestyle="--", color="orange", alpha=0.5)
        
        ax.ticklabel_format(style='sci', axis='x', scilimits=(0, 0), useMathText=True)
        ax.set_xlabel("Signal (adu)")
        ax.set_ylabel(r"$C_{10} / \mu$")
        # ax.set_xscale('log')#, linthresh=15000.0)
        # ax.set_yscale('log')#, linthresh=15.0)
        ax.set_ylim(-0.01, 0.03)
        ax.set_xlim(0, 1.0e5)
    plt.tight_layout()
    plt.savefig(f"./plots/C10_mu_{camera[ref.dataId['detector']].getId()}.png", bbox_inches='tight')
    #plt.close()


In [ ]:
for ref in refs: 
    ptc = butler.get("ptc", dataId=ref.dataId, collections=collections)
    
    fig, axes = plt.subplots(4, 4, figsize=(12,12))
    fig.suptitle(camera[ref.dataId['detector']].getName())
    axs=axes.ravel()
    for i, amp in enumerate(camera[ref.dataId['detector']].getAmplifiers()):
        ampName = amp.getName()
        ax = axs[i]
        ax.set_title(ampName)
        
        ptcTurnoff = ptc.ptcTurnoff[ampName]

        good = ptc.expIdMask[ampName] * (ptc.rawMeans[ampName] < ptcTurnoff)
        data = ptc.rawVars[ampName][good]
        model = ptc.evalPtcModel(ptc.rawMeans[ampName][good])[ampName][:,0,0]

        ax.scatter(ptc.rawMeans[ampName][good], data/model - 1, s=1)
        ax.axhline(0, linestyle="--", color="k", alpha=0.5)
        ax.axvline(ptcTurnoff, linestyle="--", color="orange", alpha=0.5)
        
        ax.ticklabel_format(style='sci', axis='x', scilimits=(0, 0), useMathText=True)
        ax.set_xlabel("Signal (adu)")
        ax.set_ylabel(r"$C_{00} / C_{00}^{model} - 1$")
        ax.set_xlim(0, 1.0e5)
    plt.tight_layout()
    plt.savefig(f"./plots/C00_residuals_{camera[ref.dataId['detector']].getId()}.png", bbox_inches='tight')
    #plt.close()


In [ ]:
for ref in refs: 
    ptc = butler.get("ptc", dataId=ref.dataId, collections=collections)
    
    fig, axes = plt.subplots(4, 4, figsize=(12,12))
    fig.suptitle(camera[ref.dataId['detector']].getName())
    axs=axes.ravel()
    for i, amp in enumerate(camera[ref.dataId['detector']].getAmplifiers()):
        ampName = amp.getName()
        ax = axs[i]
        ax.set_title(ampName)
        
        ptcTurnoff = ptc.ptcTurnoff[ampName]

        good = ptc.expIdMask[ampName] * (ptc.rawMeans[ampName] < ptcTurnoff)
        data = ptc.covariances[ampName][good][:,0,1]
        model = ptc.evalPtcModel(ptc.rawMeans[ampName][good])[ampName][:,0,1]

        ax.scatter(ptc.rawMeans[ampName][good], data/model - 1, s=1)
        ax.axhline(0, linestyle="--", color="k", alpha=0.5)
        ax.axvline(ptcTurnoff, linestyle="--", color="orange", alpha=0.5)
        
        ax.ticklabel_format(style='sci', axis='x', scilimits=(0, 0), useMathText=True)
        ax.set_xlabel("Signal (adu)")
        ax.set_ylabel(r"$C_{01} / C_{01}^{model} - 1$")
        # ax.set_xscale('log')#, linthresh=15000.0)
        # ax.set_yscale('log')#, linthresh=15.0)
        ax.set_ylim(-1, 1)
        ax.set_xlim(0, 1.0e5)
    plt.tight_layout()
    plt.savefig(f"./plots/C01_residuals_{camera[ref.dataId['detector']].getId()}.png", bbox_inches='tight')
    #plt.close()


In [ ]:
for ref in refs: 
    ptc = butler.get("ptc", dataId=ref.dataId, collections=collections)
    
    fig, axes = plt.subplots(4, 4, figsize=(12,12))
    fig.suptitle(camera[ref.dataId['detector']].getName())
    axs=axes.ravel()
    for i, amp in enumerate(camera[ref.dataId['detector']].getAmplifiers()):
        ampName = amp.getName()
        ax = axs[i]
        ax.set_title(ampName)
        
        ptcTurnoff = ptc.ptcTurnoff[ampName]

        good = ptc.expIdMask[ampName] * (ptc.rawMeans[ampName] < ptcTurnoff)
        data = ptc.covariances[ampName][good][:,1,0]
        model = ptc.evalPtcModel(ptc.rawMeans[ampName][good])[ampName][:,1,0]

        ax.scatter(ptc.rawMeans[ampName][good], data/model - 1, s=1)
        ax.axhline(0, linestyle="--", color="k", alpha=0.5)
        ax.axvline(ptcTurnoff, linestyle="--", color="orange", alpha=0.5)
        
        ax.ticklabel_format(style='sci', axis='x', scilimits=(0, 0), useMathText=True)
        ax.set_xlabel("Signal (adu)")
        ax.set_ylabel(r"$C_{10} / C_{10}^{model} - 1$")
        # ax.set_xscale('log')#, linthresh=15000.0)
        # ax.set_yscale('log')#, linthresh=15.0)
        ax.set_ylim(-1, 1)
        ax.set_xlim(0, 1.0e5)
    plt.tight_layout()
    plt.savefig(f"./plots/C10_residuals_{camera[ref.dataId['detector']].getId()}.png", bbox_inches='tight')
    #plt.close()


In [ ]:
plt.hist(np.array(a00s)*1e-6, histtype="step")
plt.ticklabel_format(style='sci', axis='x', scilimits=(0, 0), useMathText=True)
plt.xlabel(r"$a_{00}$")
plt.savefig(f"./plots/a00_hist.png", bbox_inches='tight')
#plt.close()

In [ ]:
plt.hist(np.array(n00s), histtype="step", label="PTC noise (el)")
plt.hist(np.array(empiricalN00s), histtype="step", label="Overscan read noise (el)")
plt.legend()
plt.xlabel(r"$n_{00}$")
plt.savefig(f"./plots/n00_hist.png", bbox_inches='tight')
#plt.close()

In [ ]:
plt.hist((np.array(n00s) / np.array(empiricalN00s)) - 1, histtype="step")
plt.xlabel("noise / oscanNoise - 1")
plt.savefig(f"./plots/n00_residuals_hist.png", bbox_inches='tight')
#plt.close()

In [ ]:
plt.hist(np.array(ptcTurnoffs), histtype="step")
plt.ticklabel_format(style='sci', axis='x', scilimits=(0, 0), useMathText=True)
plt.xlabel("PTC Turnoffs (adu)")
plt.savefig(f"./plots/ptc_turnoffs_hist.png", bbox_inches='tight')
#plt.close()

# 6. CTI

In [ ]:
refs = list(butler.query_datasets(
    "cti",
    where=f"instrument='LSSTCam' and detector in {str(tuple(detIds))}", 
    collections=collections
))

globalCtis = []
serialCtiTurnoffs = []
for ref in refs: 
    cti = butler.get("cti", dataId=ref.dataId, collections=collections)
    
    fig, axes = plt.subplots(4, 4, figsize=(12,12))
    fig.suptitle(camera[ref.dataId['detector']].getName())
    axs=axes.ravel()
    for i, amp in enumerate(camera[ref.dataId['detector']].getAmplifiers()):
        ampName = amp.getName()
        ax = axs[i]
        ax.set_title(ampName)
        ax.scatter(cti.signals[ampName], cti.serialEper[ampName], s=1)
        
        globalCti = cti.globalCti[ampName]
        globalCtis.append(globalCti)
        serialCtiTurnoff = cti.serialCtiTurnoff[ampName]
        serialCtiTurnoffs.append(serialCtiTurnoff)

        ax.axhline(0, linestyle="--", color="k", alpha=0.5)

        try:
            power = int(np.floor(np.log10(np.abs(globalCti))))
            num = globalCti * (10**np.abs(power))
        except:
            power=0
            num=0
            
    
        ax.axhline(globalCti, linestyle="--", color="orange", alpha=0.5, 
                   label=f"Global sCTI: {round(num, 3)}" + rf"$\times10^{{{power}}}$")
        ax.axvline(serialCtiTurnoff, linestyle="--", color="forestgreen", alpha=0.5,
                  label=f"sCTI Turnoff: {round(serialCtiTurnoff, 3)}")
        # ax.text(
        #     5e3, 4.5e4, 
        #     f"Turnoff: {round(ptcTurnoff, 1)} adu",
        # )
        # ax.text(
        #     5e3, 4.1e4, 
        #     f"Gain: {round(gain, 2)}",
        # )
        # ax.text(
        #     5e3, 3.7e4, 
        #     r"$a_{00}$: " + f"{round(a00, 2)}" + r"$\times 10^{-6}$",
        # )
        # ax.text(
        #     5e3, 3.3e4, 
        #     r"$n_{00}$: " + f"{round(n00, 2)} el",
        # )
        
        ax.ticklabel_format(style='sci', axis='both', scilimits=(0, 0), useMathText=True)
        ax.set_xlabel("Signal (el)")
        ax.set_ylabel(r"Serial EPER")
        # ax.set_xscale('log')#, linthresh=15000.0)
        # ax.set_yscale('log')#, linthresh=15.0)
        ax.set_ylim(-0.5e-5, 0.5e-5)
        ax.set_xlim(0, 2.0e5)
        ax.legend(fontsize=8)
    plt.tight_layout()
    plt.savefig(f"./plots/serialEper_{camera[ref.dataId['detector']].getId()}.png", bbox_inches='tight')
    #plt.close()


# 7. BFK

In [ ]:
refs = list(butler.query_datasets(
    "bfk",
    where=f"instrument='LSSTCam' and detector in {str(tuple(detIds))}", 
    collections=collections
))

globalCtis = []
serialCtiTurnoffs = []
for ref in refs: 
    bfk = butler.get("bfk", dataId=ref.dataId, collections=collections)
    
    fig, axes = plt.subplots(4, 4, figsize=(12,12))
    fig.suptitle(camera[ref.dataId['detector']].getName())
    axs=axes.ravel()
    for i, amp in enumerate(camera[ref.dataId['detector']].getAmplifiers()):
        ampName = amp.getName()
        ax = axs[i]
        ax.set_title(ampName + f" (Valid = {bfk.valid[ampName]})")
        length = bfk.ampKernels[ampName].shape[0]
        ax.plot(bfk.ampKernels[ampName][length//2,:], drawstyle="steps-mid", label="Serial slice")
        ax.plot(bfk.ampKernels[ampName][:,length//2], drawstyle="steps-mid", label="Parallel slice")
        ax.set_xticks([0, length//2,length-1], [-length//2 + 1 , 0,length//2 ])
        # globalCti = cti.globalCti[ampName]
        # globalCtis.append(globalCti)
        # serialCtiTurnoff = cti.serialCtiTurnoff[ampName]
        # serialCtiTurnoffs.append(serialCtiTurnoff)

        ax.axhline(0, linestyle="--", color="k", alpha=0.5)
  
        # power = int(np.floor(np.log10(np.abs(globalCti))))
        # num = globalCti * (10**np.abs(power))
    
        # ax.axhline(globalCti, linestyle="--", color="orange", alpha=0.5, 
        #            label=f"Global sCTI: {round(num, 3)}" + rf"$\times10^{{{power}}}$")
        # ax.axvline(serialCtiTurnoff, linestyle="--", color="forestgreen", alpha=0.5,
        #           label=f"sCTI Turnoff: {round(serialCtiTurnoff, 3)}")
        # ax.text(
        #     5e3, 4.5e4, 
        #     f"Turnoff: {round(ptcTurnoff, 1)} adu",
        # )
        # ax.text(
        #     5e3, 4.1e4, 
        #     f"Gain: {round(gain, 2)}",
        # )
        # ax.text(
        #     5e3, 3.7e4, 
        #     r"$a_{00}$: " + f"{round(a00, 2)}" + r"$\times 10^{-6}$",
        # )
        # ax.text(
        #     5e3, 3.3e4, 
        #     r"$n_{00}$: " + f"{round(n00, 2)} el",
        # )
        
        #ax.ticklabel_format(style='sci', axis='both', scilimits=(0, 0), useMathText=True)
        ax.set_xlabel("Offset (px)")
        ax.set_ylabel(r"(unitless)")
        # ax.set_xscale('log')#, linthresh=15000.0)
        # ax.set_yscale('log')#, linthresh=15.0)
        ax.set_ylim(-1.5e-6, 0.25e-6)
        # ax.set_xlim(0, 2.0e5)
        ax.legend(fontsize=8)
    plt.tight_layout()
    plt.savefig(f"./plots/bfk_profiles_{camera[ref.dataId['detector']].getId()}.png", bbox_inches='tight')
    #plt.close()


In [ ]:
# for amp in bfk.ampKernels.keys():
#     length = bfk.ampKernels[amp].shape[0]
#     # Less than zero +- tolerance
#     #valid = np.all(bfk.ampKernels[amp] <= 5e-9)

#     xv, yv = np.meshgrid(range(length), range(length))
#     edges = (xv < 3)
#     edges += (xv > length - 3 - 1)
#     edges += (yv < 3)
#     edges += (yv > length - 3 - 1)

#     kernelEdges = bfk.ampKernels[amp][edges].ravel()
#     std = np.std(kernelEdges)
#     valid = np.all(np.isclose(kernelEdges, 0, rtol=5*std))
    
#     kernelCenterValue = bfk.ampKernels[amp][length//2,length//2]
#     # The center should be the minimum
#     valid *= np.all(bfk.ampKernels[amp] >= kernelCenterValue) 
#     print(amp, valid)

In [ ]:
refs = list(butler.query_datasets(
    "bfk",
    where=f"instrument='LSSTCam' and detector in {str(tuple(detIds))}", 
    collections=collections
))

from matplotlib.colors import TwoSlopeNorm

globalCtis = []
serialCtiTurnoffs = []
for ref in refs: 
    bfk = butler.get("bfk", dataId=ref.dataId, collections=collections)
    
    fig, axes = plt.subplots(4, 4, figsize=(12,12))
    fig.suptitle(camera[ref.dataId['detector']].getName())
    axs=axes.ravel()
    for i, amp in enumerate(camera[ref.dataId['detector']].getAmplifiers()):
        ampName = amp.getName()
        ax = axs[i]
        ax.set_title(ampName)
        length = bfk.ampKernels[ampName].shape[0]
        ax.imshow(bfk.ampKernels[ampName], cmap="bwr", norm=TwoSlopeNorm(vcenter=0, vmin=-1.5e-6, vmax=0.1e-6))
        # ax.plot(bfk.ampKernels[ampName][:,length//2], drawstyle="steps-mid", label="Parallel slice")
        ax.set_xticks([0, length//2,length-1], [-length//2 + 1 , 0,length//2 ])
        ax.set_yticks([0, length//2,length-1], [-length//2 + 1 , 0,length//2 ])
        # globalCti = cti.globalCti[ampName]
        # globalCtis.append(globalCti)
        # serialCtiTurnoff = cti.serialCtiTurnoff[ampName]
        # serialCtiTurnoffs.append(serialCtiTurnoff)

        #ax.axhline(0, linestyle="--", color="k", alpha=0.5)
  
        # power = int(np.floor(np.log10(np.abs(globalCti))))
        # num = globalCti * (10**np.abs(power))
    
        # ax.axhline(globalCti, linestyle="--", color="orange", alpha=0.5, 
        #            label=f"Global sCTI: {round(num, 3)}" + rf"$\times10^{{{power}}}$")
        # ax.axvline(serialCtiTurnoff, linestyle="--", color="forestgreen", alpha=0.5,
        #           label=f"sCTI Turnoff: {round(serialCtiTurnoff, 3)}")
        # ax.text(
        #     5e3, 4.5e4, 
        #     f"Turnoff: {round(ptcTurnoff, 1)} adu",
        # )
        # ax.text(
        #     5e3, 4.1e4, 
        #     f"Gain: {round(gain, 2)}",
        # )
        # ax.text(
        #     5e3, 3.7e4, 
        #     r"$a_{00}$: " + f"{round(a00, 2)}" + r"$\times 10^{-6}$",
        # )
        # ax.text(
        #     5e3, 3.3e4, 
        #     r"$n_{00}$: " + f"{round(n00, 2)} el",
        # )
        
        #ax.ticklabel_format(style='sci', axis='both', scilimits=(0, 0), useMathText=True)
        # ax.set_xlabel("Offset (px)")
        # ax.set_ylabel(r"(unitless)")
        # ax.set_xscale('log')#, linthresh=15000.0)
        # ax.set_yscale('log')#, linthresh=15.0)
        # ax.set_ylim(-1.5e-6, 0.25e-6)
        # ax.set_xlim(0, 2.0e5)
        # ax.legend(fontsize=8)
    plt.tight_layout()
    plt.savefig(f"./plots/bfk_{camera[ref.dataId['detector']].getId()}.png", bbox_inches='tight')
    #plt.close()


# 8. BIAS

In [ ]:
refs = list(butler.query_datasets(
    "bias", 
    where=f"instrument='LSSTCam' and detector in {str(tuple(detIds))}", 
    collections=collections))

for ref in refs:
    plt.figure()
    img = butler.get("bias", dataId=ref.dataId, collections=collections)
    for i, amp in enumerate(camera[ref.dataId['detector']].getAmplifiers()):
        bbox = amp.getBBox()
        vals = img.image[bbox].array.ravel()
        vals = vals[(vals>-20.) * (vals<20.)]        
        _ = plt.hist(vals, bins=100, histtype='step', color=color[i], label=amp.getName())
    
    plt.axvline(0.0, linestyle="--", color="k", alpha=0.5)
    plt.yscale('log')
    plt.xlabel("adu")
    plt.legend(ncol=2)
    plt.title(f"BIAS {camera[ref.dataId['detector']].getName()}, {ref.dataId['detector']}")
    plt.savefig(f"./plots/bias_{ref.dataId['detector']}.png", bbox_inches='tight')
    #plt.close()

# 9. DARK

In [ ]:
refs = list(butler.query_datasets(
    "dark", 
    where=f"instrument='LSSTCam' and detector in {str(tuple(detIds))}", 
    collections=collections))

for ref in refs:
    plt.figure()
    img = butler.get("dark", dataId=ref.dataId, collections=collections)
    for i, amp in enumerate(camera[ref.dataId['detector']].getAmplifiers()):
        bbox = amp.getBBox()
        vals = img.image[bbox].array.ravel()
        vals = vals[(vals>-2.) * (vals<2.)]        
        _ = plt.hist(vals, bins=100, histtype='step', color=color[i], label=amp.getName())
    
    plt.axvline(0.0, linestyle="--", color="k", alpha=0.5)
    plt.yscale('log')
    plt.xlabel("adu")
    plt.legend(ncol=2)
    plt.title(f"DARK {camera[ref.dataId['detector']].getName()}, {ref.dataId['detector']}")
    # plt.savefig(f"./plots/dark_{ref.dataId['detector']}.png", bbox_inches='tight')
    # plt.close()